In [30]:
import tensorflow as tf
from tensorflow import keras
from tensorboard.plugins.hparams import api as hp_api
import kerastuner
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os
import json
import datetime
import dill

plt.rcParams["figure.figsize"] = (20, 5)

physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)  # Off when Distributed Environment

In [16]:
from tensorflow.keras.layers import TimeDistributed, Dense, Conv1D, MaxPooling1D, Bidirectional, LSTM, Dropout

In [6]:
dataset_name = "SEG_CNNLSTM"

In [4]:
timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
timestamp

'20200909-162336'

In [7]:
dataset = np.genfromtxt("data/{}_train_set.csv".format(dataset_name), delimiter="\n", dtype=np.float32) #np.int64
dataset

array([0., 0., 0., ..., 1., 3., 1.], dtype=float32)

In [33]:
# Static Parameters 
static_params = dict()

static_params["PAST_HISTORY"] = 16
static_params["FUTURE_TARGET"] = 8
static_params["BATCH_SIZE"] = 512
static_params["LOSS_FUNCTION"] = 'sparse_categorical_crossentropy'
static_params["VAL_SPLIT"] = 0.2
static_params["METRIC_ACCURACY"] = 'accuracy'
'''
word_index = np.genfromtxt("data/word_index.csv", delimiter="\n", dtype=np.int64)
vocab_size = len(word_index)
static_params["VOCAB_SIZE"] = vocab_size
'''
static_params["VOCAB_SIZE"] = 14482

In [ ]:
'''
import dill

with open("static/SparseCategoryEncoderDecoder.pkl", 'rb') as f:
    SparseCategoryEncoderDecoder = dill.load(f)

static_params["VOCAB_SIZE"] = SparseCategoryEncoderDecoder.vocab_size

with open("static/static_params.json", "w") as j :
    json.dump(static_params, j, indent=4)
'''

In [12]:
def generate_timeseries(dataset, start_index, end_index, history_size, target_size):
    data = []
    labels = []

    start_index = start_index + history_size
    if end_index is None:
        end_index = len(dataset) - target_size

    for i in range(start_index, end_index):
        indices = range(i-history_size, i)
        # Reshape data from (history_size,) to (history_size, n_feature)
        data.append(np.reshape(dataset[indices], (history_size, 1)))
        #data.append(dataset[indices])
        labels.append(np.reshape(dataset[i:i+target_size], (target_size, 1)))
        #labels.append(dataset[i:i+target_size])
    return np.array(data), np.array(labels)

In [13]:
x_train, y_train = generate_timeseries(dataset, 0, None, static_params["PAST_HISTORY"], static_params["FUTURE_TARGET"])
x_train.shape, y_train.shape

((14858, 16, 1), (14858, 8, 1))

train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.cache().batch(static_params["BATCH_SIZE"]).shuffle(static_params["BUFFER_SIZE"]).repeat()

In [14]:
def tensorboard_callback(log_dir, hist_freq=1):
    return keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=hist_freq)

In [37]:
def build_model(hp):
    model = keras.Sequential()
    model.add(Conv1D(
        filters=hp.Int("conv1d_filters", min_value=16, max_value=128, step=8), 
        kernel_size=hp.Choice("conv1d_kernel_size", values=[3, 5, 7]), padding='causal', activation='relu'
        ))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Dropout(hp.Float("conv1d_dropout", min_value=0.1, max_value=0.5, step=0.05)))
    model.add(Bidirectional(LSTM(units=hp.Int("layer_1_units", min_value=32, max_value=256, step=8), return_sequences=True)))
    model.add(Dropout(hp.Float("layer_1_dropout", min_value=0.1, max_value=0.5, step=0.05)))
    model.add(Bidirectional(LSTM(units=hp.Int("layer_2_units", min_value=32, max_value=256, step=8), return_sequences=True)))
    model.add(Dropout(hp.Float("layer_2_dropout", min_value=0.1, max_value=0.5, step=0.05)))
    model.add(TimeDistributed(Dense(static_params["VOCAB_SIZE"], activation="softmax")))
    
    model.compile(
        optimizer=keras.optimizers.Nadam(learning_rate=hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])),
        loss=static_params["LOSS_FUNCTION"],
        metrics=[static_params["METRIC_ACCURACY"]]
    )
    return model

In [38]:
tuner = kerastuner.tuners.Hyperband(
    hypermodel=build_model,
    objective='val_accuracy',
    max_epochs=100,
    factor=2,
    hyperband_iterations=3,
    #distribution_strategy=tf.distribute.MirroredStrategy(),
    tune_new_entries=True,
    directory="results",
    project_name="SEG_CNNLSTM"
)

INFO:tensorflow:Reloading Oracle from existing project results\SEG_CNNLSTM\oracle.json


In [36]:
tuner.search(
    x_train,
    y_train,
    validation_split=static_params["VAL_SPLIT"],
    epochs=100,
    callbacks=[tf.keras.callbacks.EarlyStopping('val_accuracy')] #tensorboard_callback("logs/fit/" + timestamp)
)

Epoch 1/2


ValueError: in user code:

    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:571 train_function  *
        outputs = self.distribute_strategy.run(
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:951 run  **
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2290 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2649 _call_for_each_replica
        return fn(*args, **kwargs)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:543 train_step  **
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\compile_utils.py:411 update_st

In [55]:
with open("hyper_results/SEG/tuner.pkl", 'wb') as f:
    dill.dump(tuner, f)